In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')

In [15]:
data = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
#XGBoost steps
# Label encode
# sklearn cv
# hp tuning
from sklearn.preprocessing import LabelEncoder
def make_train(data):
    train = data
    for col in train.select_dtypes(include='object'):
        le = LabelEncoder()
        train[col] = le.fit_transform(train[col])
    
    return train

In [4]:
train = make_train(data)

In [5]:
%%time
X = train.drop(['id', 'target'], axis=1)
y = train.target
data_dmatrix = xgb.DMatrix(data=X,label=y)
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=1000,early_stopping_rounds=200,metrics="rmse", as_pandas=True, seed=42)

[15:08:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[15:08:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[15:08:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
Wall time: 3min 10s


In [6]:
cv_results.tail()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
446,0.820297,0.000414,0.843728,0.000355
447,0.820245,0.000413,0.843727,0.000359
448,0.820186,0.000413,0.843721,0.000353
449,0.820151,0.000419,0.843723,0.000356
450,0.820106,0.000416,0.843715,0.000356


### 450 estimators cuz idk how to set early stopping on XGBRegressor.fit()

In [16]:
model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 450, seed=42)

In [17]:
test = make_train(test)
test_X = test.drop(['id'], axis=1)

def predict_test(model, X, y):
    model.fit(X, y)
    pred = model.predict(test_X)
#     pred.to_csv('test_pred.csv')
    return pred

In [18]:
pred = predict_test(model, X, y)
pred_df = pd.DataFrame(test['id'], columns=['id'])
pred_df['target'] = pred
pred_df.to_csv('xgb_results.csv', index=False)

[15:22:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [22]:
!kaggle competitions submit -c tabular-playground-series-feb-2021 -f xgb_results.csv -m "xgb minimal processing"

Successfully submitted to Tabular Playground Series - Feb 2021



  0%|          | 0.00/3.29M [00:00<?, ?B/s]
  0%|          | 8.00k/3.29M [00:00<00:50, 68.3kB/s]
 11%|#1        | 384k/3.29M [00:00<00:31, 96.8kB/s] 
 38%|###7      | 1.24M/3.29M [00:00<00:15, 138kB/s]
 57%|#####6    | 1.86M/3.29M [00:00<00:07, 195kB/s]
 72%|#######1  | 2.35M/3.29M [00:01<00:04, 230kB/s]
 94%|#########4| 3.10M/3.29M [00:01<00:00, 324kB/s]
100%|##########| 3.29M/3.29M [00:06<00:00, 499kB/s]
